<a href="https://colab.research.google.com/github/anirbanghoshsbi/.github.io/blob/master/experimental/Elder_Impulse_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [64]:
!pip install yfinance
!pip install pandas-ta==0.2.45b

In [76]:
import pandas as pd
import numpy as np
import yfinance as yf
import pandas_ta as ta
import warnings 
warnings.filterwarnings("ignore")

In [77]:
START = "2015-01-01"  #yyyy-mm-dd
END = '2020-05-01'

In [78]:
tker = '^nsei' #'^nsei'
nifty =yf.download(tker,start=START,end=END)
nifty=nifty.dropna()
nifty=nifty.iloc[:,:4]


[*********************100%***********************]  1 of 1 completed


ElderImpulse
Impulse System for the price bars

Bullish price bar: (13-period EMA > previous 13-period EMA) and (MACD-Histogram > previous period's MACD-Histogram)
Bearish price bar: (13-period EMA < previous 13-period EMA) and (MACD-Histogram < previous period's MACD-Histogram)
Neutral price bar: when conditions for a bullish price bar or bearish price bar are not met.

In [79]:
nifty.ta.macd(append=True)
nifty.ta.ema(length=13,append=True)
print()

In [80]:
nifty['Impulse']=0

In [81]:
for i in range(len(nifty)):
  if (nifty['EMA_13'].iloc[i]>nifty['EMA_13'].iloc[i-1] and nifty['MACDh_12_26_9'].iloc[i]>nifty['MACDh_12_26_9'].iloc[i-1]):
    nifty['Impulse'].iloc[i]=1
  elif (nifty['EMA_13'].iloc[i]<nifty['EMA_13'].iloc[i-1] and nifty['MACDh_12_26_9'].iloc[i]<nifty['MACDh_12_26_9'].iloc[i-1]):
    nifty['Impulse'].iloc[i]=-1
  else:
    nifty['Impulse'].iloc[i]=0    

In [82]:
nifty['Impulse'].value_counts()

 0    481
 1    474
-1    348
Name: Impulse, dtype: int64